In [ ]:
import requests 
import re
import pandas as pd, matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from datetime import datetime

def getdanmu(bili_url):
# 第一步：从视频页面源代码抓取cid
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"}
    mainpage = requests.get(bili_url, headers = headers)   
    ## 用正则表达式抓取cid
    cid = re.search(r"\"cid\":(\d{2,20})", mainpage.text).group(1)
    danmu_url = "https://comment.bilibili.com/{}.xml".format(cid)
    
# 第二步: 爬取弹幕
    headers_two = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"}
    danmupage = requests.get(danmu_url, headers = headers_two).content.decode('utf-8')   
    
    ##用BS来解析弹幕
    danmupage = bs(danmupage)                                  # turn the page to beautifulsoup object 
    danmulist = [] 
    for i in danmupage.find_all("d"):                          # extract all lines that looks like ----   <d p="142.54600,1,25,16777215,1623167764,0,3f873bcb,50345064242085891">逆天种法</d>
        danmulist.append(i.get("p").split(",") + [i.text])     # split the tags in p, concat with text, and append to list
    global frame
    frame = pd.DataFrame(danmulist, columns = ['seconds', 'danmu_mode', 'c', 'danmucolor_decimal', 'timestamp', 'f', 'userid(crc32)', 'h', 'danmu'])  
    frame = frame[["seconds","danmu_mode","danmucolor_decimal","timestamp",'userid(crc32)',"danmu"]]        
    
    ##整理数据
    #转换unix时间戳
    frame['date'] = frame['timestamp'].apply(lambda x: str(datetime.fromtimestamp(int(x)).date()))     
    #转换弹幕模式
    frame["danmu_mode"] = pd.to_numeric(frame["danmu_mode"])
    frame["mode"] = frame["danmu_mode"].apply(lambda x: "rolling" if x <=3 else ("bottom" if x == 4 else ("top" if x ==  5 else "backwards")))    
    #转换10进制color code为RGB
    colorlist = []
    def decimal_to_rgb(color):
        blue =  int(color) & 255
        green = (int(color) >> 8) & 255
        red =   (int(color) >> 16) & 255
        colorlist.append([blue, green, red])
    for i in frame["danmucolor_decimal"]:
        decimal_to_rgb(i)
    frame["RGB"] = colorlist
    #Round seconds
    frame["seconds"] = frame["seconds"].astype(float).astype(int)
    
    ##清除数据
    global updated
    updated = frame.drop(columns = ["danmu_mode", "timestamp", "danmucolor_decimal"])
    updated.to_excel("Danmu.xlsx")
    
#Test
url = input("请输入b站视频网址然后按回车:")
getdanmu(url)

update
